Trains and evaluate a simple MLP on the Reuters newswire topic classification task.

Adapted from: https://github.com/fchollet/keras/blob/master/examples/reuters_mlp.py

In [9]:
reset -fs

In [10]:
from keras.datasets import imdb

In [11]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5_000)

print('Data loaded.')

print(f'{len(x_train):,} train sequences')
print(f'{len(x_test):,} test sequences')

Loading data...
Data loaded.
25,000 train sequences
25,000 test sequences


In [12]:
from keras.preprocessing.text import Tokenizer

In [13]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=5_000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('Data vectorized.')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
Data vectorized.
x_train shape: (25000, 5000)
x_test shape: (25000, 5000)


In [14]:
# from keras import utils

In [15]:
# print('Convert class vector to binary class matrix '
#       '(for use with categorical_crossentropy)')
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)

# print('y_train shape:', y_train.shape)
# print('y_test shape:', y_test.shape)

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [20]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(5_000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))
print('Model built')

Building model...
Model built


In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=True,
                    validation_split=0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 20s - loss: 7.9492 - acc: 0.5014 - val_loss: 8.1689 - val_acc: 0.4876
Epoch 2/5
22500/22500 [==============================] - 20s - loss: 7.9492 - acc: 0.5014 - val_loss: 8.1689 - val_acc: 0.4876
Epoch 3/5
22500/22500 [==============================] - 20s - loss: 7.9492 - acc: 0.5014 - val_loss: 8.1689 - val_acc: 0.4876
Epoch 4/5
22500/22500 [==============================] - 21s - loss: 7.9492 - acc: 0.5014 - val_loss: 8.1689 - val_acc: 0.4876
Epoch 5/5
22500/22500 [==============================] - 20s - loss: 7.9492 - acc: 0.5014 - val_loss: 8.1689 - val_acc: 0.4876


In [23]:
score = model.evaluate(x_test, 
                       y_test,
                       batch_size=32, 
                       verbose=True)
print('Test score:', score[0])
print('Test accuracy:', score[1])

24704/25000 [============================>.] - ETA: 0sTest score: 7.97119235672
Test accuracy: 0.5
